In [1]:
import numpy as np
import pandas as pd
import umap
from sklearn.cluster import KMeans

from Celltype import *
from Utils import *
import Configurations as Configurations

/ihome/hpark/zhf16/ngs_dl/scvi-tools/python3.9/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-07 18:57:20.299150: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-07 18:57:21.736392: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-07 18:57:22.394568: W tensorflow/compiler/xla/stream_exe

In [ ]:
dirPath = "Data/";

curK = 6;

cellTypeNumericMap = {'B1':1,'B2':2,'B3':3,'B4':4,'B5':5,'B6':6,'B7':7,'B8':8,'B9':9,'B10':10};
RorNRNumericMap = {'A':1,'B':0};

resultDictionary = {"DL-Kmean":[],"Kmean":[],"DL-Leiden":[],"Leiden":[],"DL-Louvain":[],"Louvain":[]};

for index in range(1,21):#21
    print(index);
    
    expressionMatrixPath = dirPath+"SimulationLogCounts_iter_"+str(index)+".csv";
    dataset = pd.read_csv(expressionMatrixPath,header=0,index_col=0);
    
    RorNRDataPath = dirPath+"RorNRDF_iter_"+str(index)+".csv";
    RorNRData = pd.read_csv(RorNRDataPath,header=0,index_col=0,names=["RorNR"]);#,index_col=0
    RorNRData["RorNR"] = RorNRData["RorNR"].map(RorNRNumericMap,na_action=None);
    
    randomState=index;
    
    kmeans = KMeans(n_clusters=curK, random_state=randomState).fit(dataset);
    print(set(kmeans.labels_));
    silhouetteScore = metrics.silhouette_score(dataset, kmeans.labels_);
    print("Kmeans silhouetteScore: "+str(silhouetteScore));
    resultDictionary["Kmean"].append(silhouetteScore);
    
    cellTypeData = pd.DataFrame(kmeans.labels_,columns=["Celltype"]);

    XTensor = tf.convert_to_tensor(dataset);

    celltypeYTensor = tf.keras.utils.to_categorical(cellTypeData);
    celltypeYTensor = tf.convert_to_tensor(celltypeYTensor);

    RorNRYTensor = tf.keras.utils.to_categorical(RorNRData);
    RorNRYTensor = tf.convert_to_tensor(RorNRYTensor);

    trainDataset = tf.data.Dataset.from_tensor_slices((XTensor, celltypeYTensor,RorNRYTensor));
    numInputRows = XTensor.shape[0];
    numInputCols = XTensor.shape[1];
    numOutputsCellType = celltypeYTensor.shape[1];
    numOutputsRorNR = RorNRYTensor.shape[1];

    print(XTensor.shape);
    print(celltypeYTensor.shape);
    print(RorNRYTensor.shape);
    
    #Construct a model.
    celltypeModel = buildModel(numInputCols,numOutputsCellType,numOutputsRorNR);
    #Train the model.
    celltypeModel = trainPhase1Model(celltypeModel,trainDataset,Configurations.numOfEpochs,Configurations.batchSize,Configurations.celltypeLossWeight, Configurations.RorNRLossWeight);
    
    representationLayerModel = tf.keras.models.Model(inputs=celltypeModel.input,outputs=celltypeModel.get_layer("hiddenLayer3").output);
    representationLayterOutput = representationLayerModel.predict(XTensor);

    kmeans = KMeans(n_clusters=curK, random_state=0).fit(representationLayterOutput);
    print(set(kmeans.labels_));
    silhouetteScore = metrics.silhouette_score(representationLayterOutput, kmeans.labels_);
    print("DL kmeans silhouetteScore: "+str(silhouetteScore));
    resultDictionary["DL-Kmean"].append(silhouetteScore);
    
    
    adata = AnnData(dataset);
    res = sc.tl.pca(adata, n_comps=dataset.shape[1]-10);
    communities,graph, modularityScore = sce.tl.phenograph(adata, clustering_algo="leiden",copy=True,k=6,resolution_parameter=0.001);
    silhouetteScore = metrics.silhouette_score(dataset, communities);
    print("Leiden silhouetteScore: "+str(silhouetteScore));
    resultDictionary["Leiden"].append(silhouetteScore);
    
    cellTypeData = pd.DataFrame(communities,columns=["Celltype"]);
    
    XTensor = tf.convert_to_tensor(dataset);

    celltypeYTensor = tf.keras.utils.to_categorical(cellTypeData);
    celltypeYTensor = tf.convert_to_tensor(celltypeYTensor);

    RorNRYTensor = tf.keras.utils.to_categorical(RorNRData);
    RorNRYTensor = tf.convert_to_tensor(RorNRYTensor);

    trainDataset = tf.data.Dataset.from_tensor_slices((XTensor, celltypeYTensor,RorNRYTensor));
    numInputRows = XTensor.shape[0];
    numInputCols = XTensor.shape[1];
    numOutputsCellType = celltypeYTensor.shape[1];
    numOutputsRorNR = RorNRYTensor.shape[1];

    print(XTensor.shape);
    print(celltypeYTensor.shape);
    print(RorNRYTensor.shape);
    
    #Construct a model.
    celltypeModel = buildModel(numInputCols,numOutputsCellType,numOutputsRorNR);
    #Train the model.
    celltypeModel = trainPhase1Model(celltypeModel,trainDataset,Configurations.numOfEpochs,Configurations.batchSize,Configurations.celltypeLossWeight, Configurations.RorNRLossWeight);
    
    representationLayerModel = tf.keras.models.Model(inputs=celltypeModel.input,outputs=celltypeModel.get_layer("hiddenLayer3").output);
    representationLayterOutput = representationLayerModel.predict(XTensor);
    
    dframe = pd.DataFrame(representationLayterOutput);
    dframe.index, dframe.columns = (map(str, dframe.index), map(str, dframe.columns));
    adata = AnnData(dframe);
    res = sc.tl.pca(adata, n_comps=dframe.shape[1]-10);
    communities,graph, modularityScore = sce.tl.phenograph(adata, clustering_algo="leiden",copy=True,k=6,resolution_parameter=0.001);
    silhouetteScore = metrics.silhouette_score(representationLayterOutput, communities);
    print("DL leiden silhouetteScore: "+str(silhouetteScore));
    resultDictionary["DL-Leiden"].append(silhouetteScore);
    
    
    adata = AnnData(dataset);
    res = sc.tl.pca(adata, n_comps=dataset.shape[1]-10);
    communities,graph, modularityScore = sce.tl.phenograph(adata, clustering_algo="louvain",copy=True,k=6);
    silhouetteScore = metrics.silhouette_score(dataset, communities);
    print("Louvain silhouetteScore: "+str(silhouetteScore));
    resultDictionary["Louvain"].append(silhouetteScore);
    
    cellTypeData = pd.DataFrame(communities,columns=["Celltype"]);
    
    XTensor = tf.convert_to_tensor(dataset);

    celltypeYTensor = tf.keras.utils.to_categorical(cellTypeData);
    celltypeYTensor = tf.convert_to_tensor(celltypeYTensor);

    RorNRYTensor = tf.keras.utils.to_categorical(RorNRData);
    RorNRYTensor = tf.convert_to_tensor(RorNRYTensor);

    trainDataset = tf.data.Dataset.from_tensor_slices((XTensor, celltypeYTensor,RorNRYTensor));
    numInputRows = XTensor.shape[0];
    numInputCols = XTensor.shape[1];
    numOutputsCellType = celltypeYTensor.shape[1];
    numOutputsRorNR = RorNRYTensor.shape[1];

    print(XTensor.shape);
    print(celltypeYTensor.shape);
    print(RorNRYTensor.shape);
    
    #Construct a model.
    celltypeModel = buildModel(numInputCols,numOutputsCellType,numOutputsRorNR);
    #Train the model.
    celltypeModel = trainPhase1Model(celltypeModel,trainDataset,Configurations.numOfEpochs,Configurations.batchSize,Configurations.celltypeLossWeight, Configurations.RorNRLossWeight);
    
    representationLayerModel = tf.keras.models.Model(inputs=celltypeModel.input,outputs=celltypeModel.get_layer("hiddenLayer3").output);
    representationLayterOutput = representationLayerModel.predict(XTensor);
    
    dframe = pd.DataFrame(representationLayterOutput);
    dframe.index, dframe.columns = (map(str, dframe.index), map(str, dframe.columns));
    adata = AnnData(dframe);
    res = sc.tl.pca(adata, n_comps=dframe.shape[1]-10);
    communities,graph, modularityScore = sce.tl.phenograph(adata, clustering_algo="louvain",copy=True,k=6,resolution_parameter=0.001);
    silhouetteScore = metrics.silhouette_score(representationLayterOutput, communities);
    print("DL louvain silhouetteScore: "+str(silhouetteScore));
    resultDictionary["DL-Louvain"].append(silhouetteScore);

1


/ihome/hpark/zhf16/ngs_dl/scvi-tools/python3.9/lib/python3.9/site-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


{0, 1, 2, 3, 4, 5}
Kmeans silhouetteScore: 0.5145287821708776
(10000, 1000)
(10000, 6)
(10000, 2)


2023-03-07 18:58:34.375861: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /ihome/hpark/zhf16/ngs_dl/scvi-tools/python3.9/lib:
2023-03-07 18:58:34.375900: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-07 18:58:34.375922: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (htc-1024-n0.crc.pitt.edu): /proc/driver/nvidia/version does not exist
2023-03-07 18:58:34.380337: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with t


Start of epoch 1/60
Training accuracy for Celltype: 0.7479, accuracy for RorNR: 0.8390.

Start of epoch 2/60
Training accuracy for Celltype: 0.8870, accuracy for RorNR: 0.8420.

Start of epoch 3/60
Training accuracy for Celltype: 0.8939, accuracy for RorNR: 0.8433.

Start of epoch 4/60
Training accuracy for Celltype: 0.9285, accuracy for RorNR: 0.8550.

Start of epoch 5/60
Training accuracy for Celltype: 0.9385, accuracy for RorNR: 0.8622.

Start of epoch 6/60
Training accuracy for Celltype: 0.9510, accuracy for RorNR: 0.8644.

Start of epoch 7/60
Training accuracy for Celltype: 0.9566, accuracy for RorNR: 0.8636.

Start of epoch 8/60
Training accuracy for Celltype: 0.9426, accuracy for RorNR: 0.8625.

Start of epoch 9/60
Training accuracy for Celltype: 0.9607, accuracy for RorNR: 0.8651.

Start of epoch 10/60
Training accuracy for Celltype: 0.9611, accuracy for RorNR: 0.8662.

Start of epoch 11/60
Training accuracy for Celltype: 0.9689, accuracy for RorNR: 0.8588.

Start of epoch 12/

...

{0, 1, 2, 3, 4, 5}
DL kmeans silhouetteScore: 0.74639034


/scratch/slurm-1532189/ipykernel_12704/1810950256.py:63: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = AnnData(dataset);


Finding 6 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 0.7347426414489746 seconds
Jaccard graph constructed in 1.3323125839233398 seconds
Running Leiden optimization
Leiden completed in 0.07309746742248535 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 3.123549222946167 seconds
Leiden silhouetteScore: -0.0660726358644382
(10000, 1000)
(10000, 7)
(10000, 2)

Start of epoch 1/60
Training accuracy for Celltype: 0.5689, accuracy for RorNR: 0.8382.

Start of epoch 2/60
Training accuracy for Celltype: 0.6894, accuracy for RorNR: 0.8491.

Start of epoch 3/60
Training accuracy for Celltype: 0.7409, accuracy for RorNR: 0.8609.

Start of epoch 4/60
Training accuracy for Celltype: 0.7569, accuracy for RorNR: 0.8627.

Start of epoch 5/60
Training accuracy for Celltype: 0.7728, accuracy for RorNR: 0.8617.

Start of epoch 6/60
Training accuracy for Celltype: 0.7845, accuracy for RorNR: 0.8576.

Start of epoch 7/60
Training accuracy

/scratch/slurm-1532189/ipykernel_12704/1810950256.py:108: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata = AnnData(dataset);


Finding 6 nearest neighbors using minkowski metric and 'auto' algorithm
Neighbors computed in 0.7466020584106445 seconds
Jaccard graph constructed in 1.4997923374176025 seconds
Wrote graph to binary file in 0.017091035842895508 seconds
Running Louvain modularity optimization
After 1 runs, maximum modularity is Q = 0.887942
After 5 runs, maximum modularity is Q = 0.888996
Louvain completed 25 runs in 2.573870897293091 seconds
Sorting communities by size, please wait ...
PhenoGraph completed in 6.012351751327515 seconds
Louvain silhouetteScore: 0.02699417862549373
(10000, 1000)
(10000, 17)
(10000, 2)

Start of epoch 1/60
Training accuracy for Celltype: 0.4431, accuracy for RorNR: 0.8371.

Start of epoch 2/60
Training accuracy for Celltype: 0.5706, accuracy for RorNR: 0.8478.

Start of epoch 3/60
Training accuracy for Celltype: 0.6427, accuracy for RorNR: 0.8663.

Start of epoch 4/60
Training accuracy for Celltype: 0.6288, accuracy for RorNR: 0.8641.

Start of epoch 5/60
Training accuracy

In [ ]:
pd.DataFrame(resultDictionary).to_csv("resultDictionary2.csv");